In [2]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

from TropiGAT_functions import *
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_1908"


> pre-processing

In [3]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

> run the predictions 

In [4]:
# Run the predictions : 
graph_test = graph_single_Dpo_pred(dpo_embeddings)
r_pred = run_predictions(graph_test,dpo_embeddings)
formatted_pred = format_predictions(r_pred)


In [15]:
cleaner_pred = get_top_n_kltypes(formatted_pred , 20)
clean_print(cleaner_pred)

{'K10PH82C1': [('KL18', 4.5750237), ('KL71', 3.6657069), ('KL82', 2.9483984), ('KL159', 2.7621887), ('KL128', 2.754026), ('KL157', 2.66852),
               ('KL70', 2.6322627), ('KL109', 2.5213988), ('KL66', 1.9797106), ('KL170', 1.9715576), ('KL119', 1.9421154), ('KL115', 1.9404724),
               ('KL140', 1.9174399), ('KL113', 1.7949989), ('KL139', 1.780188), ('KL46', 1.7707273), ('KL60', 1.697552), ('KL67', 1.6673003),
               ('KL6', 1.6665869), ('KL30', 1.5164015)],
 'K11PH164C1': [('KL119', 4.900301), ('KL145', 3.711521), ('KL139', 3.2529707), ('KL127', 3.0780547), ('KL170', 2.9044845), ('KL56', 2.6073728),
                ('KL159', 2.5697932), ('KL128', 2.2303894), ('KL70', 1.9315317), ('KL141', 1.8951347), ('KL33', 1.8278246), ('KL164', 1.7952178),
                ('KL29', 1.7568667), ('KL41', 1.459492), ('KL82', 1.4026144), ('KL53', 1.2327967), ('KL130', 0.98795766), ('KL148', 0.986772),
                ('KL115', 0.9576779), ('KL12', 0.9328747)],
 'K13PH07C1S': [('KL1

In [13]:
KL_types_L = [f"KL{n}" for n in [102,107,64,112,151,17,24,25,27,2]]

spot_test = {}
for kl in KL_types_L :
    phages_to_spot = []
    for phage in cleaner_pred :
        shots = [item[0] for item in cleaner_pred[phage]]
        if kl in shots :
            phages_to_spot.append(phage)
    spot_test[kl] = phages_to_spot

In [14]:
clean_print(spot_test)

{'KL102': [],
 'KL107': [],
 'KL112': ['K24PH164C1', 'K54lambda1_1_1'],
 'KL151': ['K21lambda1'],
 'KL17': ['K17alfa62'],
 'KL2': ['K48PH164C1', 'K2PH164C2'],
 'KL24': ['K24PH164C1', 'K23PH08C2', 'K27PH129C1', 'K54lambda1_1_1'],
 'KL25': ['K22PH164C1', 'K44PH129C1', 'K25PH129C1'],
 'KL27': ['K5lambda5', 'K41P2', 'K38PH09C2', 'K65PH164', 'K6PH25C3', 'K53PH164C2', 'K27PH129C1'],
 'KL64': ['K14PH164C1']}
